In [5]:
import cv2
import mediapipe as mp
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return angle if angle <= 180 else 360 - angle

# Function to extract posture features and labels from annotated images
def extract_features_and_labels(correct_dataset_path, incorrect_dataset_path):
    features = []
    labels = []
    
    # Process correct posture images
    for file_name in os.listdir(correct_dataset_path):
        file_path = os.path.join(correct_dataset_path, file_name)
        if file_name.endswith(('.jpg', '.png', '.jpeg')):
            image = cv2.imread(file_path)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image_rgb)

            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                height, width, _ = image.shape
                
                # Extract key points and calculate angles
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * width,
                                 landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * height]
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x * width,
                                  landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * height]
                left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x * width,
                              landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y * height]
                right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x * width,
                               landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y * height]
                left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x * width,
                              landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y * height]
                right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x * width,
                               landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y * height]

                # Calculate arm angles
                left_arm_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                right_arm_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

                # Check posture correctness
                x_axis_aligned = abs(left_shoulder[1] - right_shoulder[1]) < 20
                arm_angle_correct = 90 <= left_arm_angle <= 180 and 90 <= right_arm_angle <= 180
                posture_correct = x_axis_aligned and arm_angle_correct

                # Create feature vector (arm angles, alignment)
                feature_vector = [left_arm_angle, right_arm_angle, int(x_axis_aligned), int(arm_angle_correct)]
                features.append(feature_vector)
                labels.append(1)  # Label for correct posture

    # Process incorrect posture images
    for file_name in os.listdir(incorrect_dataset_path):
        file_path = os.path.join(incorrect_dataset_path, file_name)
        if file_name.endswith(('.jpg', '.png', '.jpeg')):
            image = cv2.imread(file_path)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(image_rgb)

            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                height, width, _ = image.shape
                
                # Extract key points and calculate angles for incorrect postures as well
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x * width,
                                 landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * height]
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x * width,
                                  landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * height]
                left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x * width,
                              landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y * height]
                right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x * width,
                               landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y * height]
                left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x * width,
                              landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y * height]
                right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x * width,
                               landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y * height]

                # Calculate arm angles for incorrect postures as well
                left_arm_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                right_arm_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

                # Check posture correctness (this will likely be false for incorrect postures)
                x_axis_aligned = abs(left_shoulder[1] - right_shoulder[1]) < 20
                arm_angle_correct = 90 <= left_arm_angle <= 180 and 90 <= right_arm_angle <= 180

                # Create feature vector (arm angles, alignment)
                feature_vector = [left_arm_angle, right_arm_angle, int(x_axis_aligned), int(arm_angle_correct)]
                features.append(feature_vector)
                labels.append(0)  # Label for incorrect posture

    return np.array(features), np.array(labels)

# Paths to your annotated datasets (adjust as necessary)
correct_dataset_path = r"D:\ml final\bicep_annotated"  # Path for correct postures
incorrect_dataset_path = r"D:\ml final\incorrect annotated"  # Path for incorrect postures

# Extract features and labels from both datasets
features, labels = extract_features_and_labels(correct_dataset_path, incorrect_dataset_path)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Build the neural network model using Keras
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),  # Input layer
    tf.keras.layers.Dense(32, activation='relu'),  # Hidden layer
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer (binary classification)
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with epochs (e.g., 50 epochs)
epochs = 50
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs)

# Evaluate the model on the test set and print metrics
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Make predictions and generate classification report and confusion matrix
y_pred_probabilities = model.predict(X_test).flatten()
y_pred_classes = (y_pred_probabilities > 0.5).astype(int)  # Convert probabilities to class labels

print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_classes))

# Save the trained model with the specified name
model_filename = "23asdas_bicep_model.h5"
model.save(model_filename)  # Save the model in HDF5 format
print(f"Model saved as: {model_filename}")

c:\Users\Pruthviraj H Jagtap\AppData\Local\Programs\Python\Python310\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Epoch 1/50


c:\Users\Pruthviraj H Jagtap\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6477 - loss: 0.9952 - val_accuracy: 0.7600 - val_loss: 0.5297
Epoch 2/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7491 - loss: 0.5233 - val_accuracy: 0.7575 - val_loss: 0.5018
Epoch 3/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7506 - loss: 0.5167 - val_accuracy: 0.7475 - val_loss: 0.5289
Epoch 4/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7721 - loss: 0.5002 - val_accuracy: 0.7550 - val_loss: 0.4918
Epoch 5/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7669 - loss: 0.4900 - val_accuracy: 0.7525 - val_loss: 0.4992
Epoch 6/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7663 - loss: 0.4769 - val_accuracy: 0.7450 - val_loss: 0.4879
Epoch 7/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7775 - loss: 0.4547 - val_accuracy: 0.7575 - val_loss: 0.4645
Epoch 8/50
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7743 - loss: 0.4591 - val_accuracy: 0.7725 - val_loss: 0.4756
Epo


Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.71      0.75       140
           1       0.85      0.90      0.88       260

    accuracy                           0.83       400
   macro avg       0.82      0.81      0.81       400
weighted avg       0.83      0.83      0.83       400


Confusion Matrix:
[[ 99  41]
 [ 25 235]]
Model saved as: 23asdas_bicep_model.h5


In [3]:
from sklearn.metrics import classification_report, confusion_matrix

# Evaluate the model
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy: {accuracy * 100:.2f}%")

# Calculate and print precision, recall, F1 score
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Incorrect', 'Correct']))

# Generate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(conf_matrix)

Model accuracy: 92.00%

Classification Report:
              precision    recall  f1-score   support

   Incorrect       0.88      0.90      0.89       140
     Correct       0.95      0.93      0.94       260

    accuracy                           0.92       400
   macro avg       0.91      0.92      0.91       400
weighted avg       0.92      0.92      0.92       400


Confusion Matrix:
[[126  14]
 [ 18 242]]


In [8]:
import cv2
import mediapipe as mp
import numpy as np
import time
from datetime import datetime
import tensorflow as tf

# Load the trained posture detection model
model = tf.keras.models.load_model("23asdas_bicep_model.h5")  # Adjust the path if necessary

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First point
    b = np.array(b)  # Mid point
    c = np.array(c)  # End point

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

# Function to draw feedback on the frame
def draw_feedback(frame, landmarks, color_feedback):
    for landmark in landmarks:
        cv2.circle(frame, (int(landmark[0] * frame.shape[1]), int(landmark[1] * frame.shape[0])), 
                       10, color_feedback, -1)

# Function to save session data to a text file
def save_session_data(rep_count, elapsed_time, start_time_str):
    with open('session_data.txt', 'a') as f:  # Open in append mode
        f.write(f"Repetitions: {rep_count}\n")
        f.write(f"Duration: {elapsed_time}\n")
        f.write(f"Start Time: {start_time_str}\n")
        f.write("\n")  # Add a newline for separation between sessions
    print("Session data saved.")

# Start capturing video from webcam
cap = cv2.VideoCapture(0)

# Performance logging variables
rep_count = 0
in_rep = False  # Flag to track whether we are in a repetition
session_start_time = time.time()  # Start timing immediately when the session begins

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the image to RGB
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        # Draw landmarks on the image
        mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Extract joint positions (shoulder, elbow, wrist)
        landmarks = results.pose_landmarks.landmark
        
        left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
        
        left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
        
        right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                         landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
        right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
        
        right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

        # Calculate angles for standing bicep curl (shoulder-elbow-wrist)
        elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)

        # Calculate shoulder alignment angle (shoulder-hip-elbow)
        shoulder_hip_angle = calculate_angle(left_hip, left_shoulder, left_elbow)

        # Prepare input for model prediction (ensure it matches training features)
        input_data = np.array([[elbow_angle, shoulder_hip_angle,
                                 int(abs(left_shoulder[1] - right_shoulder[1]) < 20), 
                                 int(90 <= elbow_angle <= 180 and 90 <= shoulder_hip_angle <= 180)]])  
        
        # Make prediction using the loaded model
        prediction_probabilities = model.predict(input_data)
        predicted_class = (prediction_probabilities > 0.5).astype(int)[0][0]  # Binary classification
        
        feedback = ""
        color_feedback = (255, 255, 255)  # Default color for neutral feedback

        if predicted_class == 1:  # Correct posture predicted
            feedback += "Good posture!\n"
            color_feedback = (0, 255, 0)  # Green for correct posture
            
            if elbow_angle < 15 and shoulder_hip_angle <= 20 and not in_rep:
                rep_count += 1
                in_rep = True
                
        else:  # Incorrect posture predicted
            feedback += "Incorrect posture!\n"
            color_feedback = (0, 0, 255)  # Red for incorrect posture
            in_rep = False
            
            if elbow_angle > 175:
                feedback += "Start position\n"
            elif elbow_angle < 15 and shoulder_hip_angle <= 20:
                feedback += "End position\n"

        # Draw circles around key points for visual feedback
        draw_feedback(frame, [left_shoulder, left_elbow, left_wrist,
                              right_shoulder, right_elbow, right_wrist], color_feedback)

        # Create a semi-transparent background for feedback text
        overlay = frame.copy()
        cv2.rectangle(overlay, (10, 10), (400, 150), (50, 50, 50), -1) 
        alpha = 0.5 
        cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

        # Display feedback on the frame with a background rectangle
        cv2.putText(frame, feedback.strip(), (20, 40), cv2.FONT_HERSHEY_SIMPLEX,
                    1.0, (255, 255, 255), 2)

    else:
        cv2.putText(frame, "No pose detected", (10, frame.shape[0] // 2), 
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1.5,(0, 0, 255), 3)

    # Show repetitions count at the top right corner
    cv2.putText(frame, f"Reps: {rep_count}", (frame.shape[1] - 250, 30), 
                cv2.FONT_HERSHEY_SIMPLEX,
                1,(255 ,0 ,0),2)

    elapsed_time = time.time() - session_start_time
    
    # Show session duration formatted as HH:MM:SS if session has started
    elapsed_time_str = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
    session_info = f"Time: {elapsed_time_str}"
    cv2.putText(frame, session_info,(10 ,frame.shape[0]-10),
                cv2.FONT_HERSHEY_SIMPLEX ,1,(255 ,255 ,255),3)

    # Show average speed of repetitions per minute if reps > 0 and session has started
    if rep_count > 0:
        avg_speed_per_minute = rep_count / elapsed_time * 60
        speed_info = f"Avg Speed: {avg_speed_per_minute:.1f} reps/min"
        cv2.putText(frame,speed_info,(10 ,frame.shape[0]-50),
                    cv2.FONT_HERSHEY_SIMPLEX ,1,(200 ,200 ,200),3)

    # Show the output frame
    cv2.imshow('Bicep Curl Posture Detection', frame)

    key = cv2.waitKey(10)
    if key == ord('q'):   # Press 'q' to quit.
        break

# Save session data when exiting the loop as plain text file.
elapsed_time_final_str = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
start_time_str = datetime.now().strftime("%H:%M")   # Get current local time in HH:MM format.
save_session_data(rep_count, elapsed_time_final_str, start_time_str)

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━